In [1]:
import sys
!{sys.executable} -m pip install yfinance as yf
!{sys.executable} -m pip install pandas as pd

ERROR: Could not find a version that satisfies the requirement as (from versions: none)
ERROR: No matching distribution found for as
ERROR: Could not find a version that satisfies the requirement as (from versions: none)
ERROR: No matching distribution found for as


In [2]:
from locale import D_FMT
import yfinance as yf
import pandas as pd
from sqlalchemy import create_engine
import logging

In [17]:
def getInfo(ticker: str):
    """getInfo creates a Ticker object, then returns a pandas df of all the available info
            
        Parameters:
                ticker: String
                
        Returns:
                A Pandas DataFrame
    
    """
    keys = []
    values = []
    yfTicker = yf.Ticker(ticker.upper())
    info = yfTicker.info
    keys = info.keys()
    values = info.values()
    zipped = list(zip(keys,values))

    zipped.pop(149) # removes logo url to avoid SQL error
    zipped.pop(8) # removes website url to avoid SQL error
    zipped.pop(3) # removes longBusinessSummary to avoid SQL error
    
    df = pd.DataFrame(data=zipped, columns=['keys','values'])
    
    return df


def dictToDF(dictionary: dict):
    """The dictToDF method takes in a dictionary and returns a Pandas DataFrame
    
        Parameters:
                ticker: String
                
        Returns:
                A Pandas DataFrame
                
    """
    columns = ["keys","values"]
    keys = []
    values = []
    zipped = []
    
    # Separates keys from values
    keys = dictionary.keys()
    
    # Separates values from keys
    values = dictionary.values()
    
    # Zips the keys and values into a list of tuples for easy viewing
    zipped = zip(keys,values)
    
    # Pandas DataFrame of all info from ticker
    df = pd.DataFrame(zipped,columns=columns)
    
    return df


def generalInfo(ticker: str):
    """
    The generalInfo method takes in any Ticker and returns the general info associated with that ticker
    
        Parameters:
                ticker: String
                
        Returns:
                A Pandas DataFrame
    """
    key_list = ["longName","symbol","market","industry","sector","country",
                "state","city","address1","zip","phone","website"]
    general_info = {}
    
    tick = yf.Ticker(ticker)
    info = tick.info
    
    for key in key_list:
        for i in info:
            if i == key:
                general_info[key] = info[i]
                
    general = dictToDF(general_info)
    
    return general


def getFinancials(ticker: str):
    """getFinancials returns a dataframe with a lot of useful financial information
    
        Parameters:
                ticker: String
                
        Returns:
                A Pandas DataFrame of the ticker's financial information
    """
    ticker = yf.Ticker(ticker.upper())
    bs = ticker.balancesheet
    cf = ticker.cashflow
    fn = ticker.financials
    
    return pd.concat([bs,cf,fn])


def toSQL(engine, tableName, df):
    """toSQL creates a table in a MySQL server to store a given Pandas DataFrame

        Parameters:
                engine: A SQL Engine to connect to MySQL server
                tableName: Name of the table being created in MySQL server
                df: A Pandas DataFrame that consists of a user-selected choice
    """
    logging.basicConfig(filename='logfile.log', encoding='utf-8', level=logging.DEBUG)
    dbConnection = engine.connect()

    try:
        df.to_sql(tableName, dbConnection)
    except ValueError as vx:
        print(vx)
    except Exception as ex:   
        print(ex)
    else:
        print("Table %s created successfully."%tableName);   
    finally:
        dbConnection.close()


def getHistoricPrice(ticker: str):
    """getHistoricPrice returns a dataframe with the historic stock price of the given ticker
    
        Parameters:
                ticker: String
                
        Returns:
                A Pandas DataFrame of the ticker's historic stock price
    """
    ticker = yf.Ticker(ticker.upper())
    hist = ticker.history(period='max')
    df = pd.DataFrame(hist)

    return df


def getDividends(ticker: str):
    """getDividends returns a dataframe with the all the dividends paid out for the given ticker
    
        Parameters:
                ticker: String
                
        Returns:
                A Pandas DataFrame of the ticker's historic stock price
    """
    ticker = yf.Ticker(ticker.upper())
    div = ticker.dividends
    df = pd.DataFrame(div)

    return df

In [4]:
getInfo("MSFT")

,keys,values
0,zip,98052-6399
1,sector,Technology
2,fullTimeEmployees,181000
3,city,Redmond
4,phone,425 882 8080
...,...,...
145,bidSize,2900
146,dayHigh,263.28
147,preMarketPrice,260
148,logo_url,https://logo.clearbit.com/microsoft.com


In [6]:
# List of keys to pick from for all info
tick = yf.Ticker("AMZN")
info = tick.info
this = info.keys()

for i in this:
    print(i)

zip
sector
fullTimeEmployees
longBusinessSummary
city
phone
state
country
companyOfficers
website
maxAge
address1
industry
ebitdaMargins
profitMargins
grossMargins
operatingCashflow
revenueGrowth
operatingMargins
ebitda
targetLowPrice
recommendationKey
grossProfits
freeCashflow
targetMedianPrice
currentPrice
earningsGrowth
currentRatio
returnOnAssets
numberOfAnalystOpinions
targetMeanPrice
debtToEquity
returnOnEquity
targetHighPrice
totalCash
totalDebt
totalRevenue
totalCashPerShare
financialCurrency
revenuePerShare
quickRatio
recommendationMean
exchange
shortName
longName
exchangeTimezoneName
exchangeTimezoneShortName
isEsgPopulated
gmtOffSetMilliseconds
quoteType
symbol
messageBoardId
market
annualHoldingsTurnover
enterpriseToRevenue
beta3Year
enterpriseToEbitda
52WeekChange
morningStarRiskRating
forwardEps
revenueQuarterlyGrowth
sharesOutstanding
fundInceptionDate
annualReportExpenseRatio
totalAssets
bookValue
sharesShort
sharesPercentSharesOut
fundFamily
lastFiscalYearEnd
heldPerce

In [18]:
generalInfo("aapl")

,keys,values
0,longName,Apple Inc.
1,symbol,AAPL
2,market,us_market
3,industry,Consumer Electronics
4,sector,Technology
5,country,United States
6,state,CA
7,city,Cupertino
8,address1,One Apple Park Way
9,zip,95014


In [12]:
getFinancials("nflx")

,2021-12-31,2020-12-31,2019-12-31,2018-12-31
Intangible Assets,13799221000.0,13747607000.0,14703352000.0,14951141000.0
Total Liab,28735415000.0,28215119000.0,26393555000.0,20735635000.0
Total Stockholder Equity,15849248000.0,11065240000.0,7582157000.0,5238765000.0
Other Current Liab,5502309000.0,5547528000.0,5338306000.0,5442461000.0
Total Assets,44584663000.0,39280359000.0,33975712000.0,25974400000.0
Common Stock,4024561000.0,3447698000.0,2793929000.0,2315988000.0
Other Current Assets,913883000.0,742169000.0,524669000.0,5151186000.0
Retained Earnings,12689372000.0,7573144000.0,4811749000.0,2942359000.0
Other Liab,3144891000.0,2654608000.0,3355987000.0,3888257000.0
Treasury Stock,-864685000.0,44398000.0,-23521000.0,-19582000.0


In [13]:
getHistoricPrice("hd")

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1981-09-22,0.018414,0.018414,0.018414,0.018414,7449469,0.0,0.0
1981-09-23,0.018414,0.018414,0.018414,0.018414,7381125,0.0,0.0
1981-09-24,0.018414,0.018414,0.018414,0.018414,5091609,0.0,0.0
1981-09-25,0.018414,0.018414,0.018414,0.018414,7312781,0.0,0.0
1981-09-28,0.018414,0.018414,0.018414,0.018414,5809219,0.0,0.0
...,...,...,...,...,...,...,...
2022-05-10,298.459991,301.070007,287.660004,291.160004,4392900,0.0,0.0
2022-05-11,291.450012,293.329987,282.570007,282.959991,3880500,0.0,0.0
2022-05-12,281.570007,293.179993,280.630005,289.690002,4447700,0.0,0.0


In [14]:
getDividends("o")

,Dividends
Date,
1994-10-26,0.072674
1994-11-25,0.072674
1994-12-27,0.072674
1995-01-26,0.072674
1995-02-23,0.072674
...,...
2021-12-31,0.247000
2022-01-31,0.247000
2022-02-28,0.247000
